In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import mlflow
import os
import requests
from sklearn.metrics import f1_score
import pandas as pd
pd.set_option('display.max_columns', None)

import {{cookiecutter.project_dir}}.helpers.helpers as helpers
wine_dataset_df = helpers.get_wine_dataset()

#### Search Runs

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000/")
runs = mlflow.search_runs(experiment_ids=['1'])
display(runs)
runs['metrics.val_f1_score']

#### Get Ordered Runs

In [ ]:
ordered_runs = mlflow.search_runs(experiment_ids=['1'], order_by=["metrics.val_f1_score DESC"])
display(ordered_runs)
ordered_runs['metrics.val_f1_score']

#### Make Inference with Selected Model

In [ ]:
which_run = 4
model_uri = f"runs:/{ordered_runs.iloc[which_run, :].run_id}/train/model"
print(model_uri)
model = mlflow.pyfunc.load_model(model_uri)
y_true = wine_dataset_df['is_red']
y_pred = model.predict(wine_dataset_df)
print(f'f1_score: {f1_score(y_true, y_pred)}')

#### Make Inference with Request to REST API

In [ ]:
url = 'http://127.0.0.1:5001/invocations'

headers = {
    "Content-Type": "application/json"
}
json_data = wine_dataset_df.drop(columns=['is_red']).to_dict(orient='records')
payload = {
    'dataframe_records': json_data
}

response = requests.post(url, json=payload, headers=headers)

print(response.status_code)
print(response.json())

y_pred_rest_api = response.json()['predictions']
print(f'f1_score: {f1_score(y_true, y_pred_rest_api)}')